In [1]:
import db
import pickle
import joblib
import numpy as np
from random import choice, choices, randint, randrange, random, sample, shuffle
from sklearn.utils import shuffle
from keras.models import load_model

model_dnn = load_model('./models/dnn.h5')
# model_rf = joblib.load("./models/rf.joblib")
dbConn = db.DataBase()
gods = dbConn.select("SELECT * FROM God ORDER BY name", ())
matches = dbConn.select("SELECT "
                        "t1_god1, t1_god2, t1_god3, t1_god4, t1_god5, "
                        "t2_god1, t2_god2, t2_god3, t2_god4, t2_god5, "
                        "ban1, ban2, ban3, ban4, ban5, ban6, ban7, ban8, ban9, ban10, "
                        "win FROM Match", ())

In [2]:
def get_win_good_with():
    # synergy win rate
    gods_ids = list(map(lambda x: x[0], gods))
    good_with = np.zeros((len(gods_ids), len(gods_ids))) * 0.5
    good_with_num_matches = np.zeros((len(gods_ids), len(gods_ids)))
    for match in matches:
        for god_1_id in match[:5]:
            i = gods_ids.index(god_1_id)
            for god_2_id in match[:5]:
                if god_1_id != god_2_id:
                    j = gods_ids.index(god_2_id)
                    good_with[i, j] += 1 if match[len(match) - 1] == 1 else 0
                    good_with_num_matches[i, j] += 1

        for god_1_id in match[5:10]:
            i = gods_ids.index(god_1_id)
            for god_2_id in match[5:10]:
                if god_1_id != god_2_id:
                    j = gods_ids.index(god_2_id)
                    good_with[i, j] += 1 if match[len(match) - 1] == 2 else 0
                    good_with_num_matches[i, j] += 1

    win_good_with = np.divide(good_with, good_with_num_matches, out=np.zeros_like(good_with), where=good_with_num_matches != 0)
    pickle.dump(win_good_with, open("./pickles/win_good_with.p", "wb"))

    return win_good_with


def get_win_good_against():
    # countering win rate
    gods_ids = list(map(lambda x: x[0], gods))
    good_against = np.zeros((len(gods_ids), len(gods_ids))) * 0.5
    good_against_num_matches = np.zeros((len(gods_ids), len(gods_ids)))
    for match in matches:
        for god_1_id in match[:5]:
            i = gods_ids.index(god_1_id)
            for god_2_id in match[5:10]:
                j = gods_ids.index(god_2_id)

                if match[len(match) - 1] == 1:
                    good_against[i, j] += 1
                else:
                    good_against[j, i] += 1

                good_against_num_matches[i, j] += 1
                good_against_num_matches[j, i] += 1

    win_good_against = np.divide(good_against, good_against_num_matches, out=np.zeros_like(good_against), where=good_against_num_matches != 0)
    pickle.dump(win_good_against, open("./pickles/win_good_against.p", "wb"))

    return win_good_against

win_good_with = get_win_good_with()
win_good_against = get_win_good_against()

In [3]:
def get_synergy_counter(gods_team_1, gods_team_2):
    team_1 = []
    team_2 = []
    for i in range(len(gods_team_1)):
        if gods_team_1[i] == 1:
            team_1.append(gods[i][1])
    for i in range(len(gods_team_2)):
        if gods_team_2[i] == 1:
            team_2.append(gods[i][1])
            
    if len(team_1) < 2 or len(team_2) < 2:
        return 0, 0
    
    gods_names = list(map(lambda x: x[1], gods))
    s_team_1 = 0
    s_team_2 = 0
    n_synergy_team_1 = 0
    n_synergy_team_2 = 0
    counter = 0
    n_counter = 0
    for god_1_name in team_1:
        i = gods_names.index(god_1_name)
        for god_2_name in team_1:
            if god_1_name != god_2_name:
                j = gods_names.index(god_2_name)
                s_team_1 += win_good_with[i, j] if win_good_with[i, j] > 0 else 0.5
                n_synergy_team_1 += 1

    for god_1_name in team_2:
        i = gods_names.index(god_1_name)
        for god_2_name in team_2:
            if god_1_name != god_2_name:
                j = gods_names.index(god_2_name)
                s_team_2 += win_good_with[i, j] if win_good_with[i, j] > 0 else 0.5
                n_synergy_team_2 += 1

        for god_2_name in team_1:
            if god_1_name != god_2_name:
                j = gods_names.index(god_2_name)
                counter += win_good_against[i, j]
                n_counter += 1

    s_team_1 = s_team_1 / n_synergy_team_1
    s_team_2 = s_team_2 / n_synergy_team_2
    synergy = s_team_1 / (s_team_1 + s_team_2)
    return synergy, counter / n_counter

def get_god_selection(team_1, team_2):
    gods_team_1 = list(map(lambda god: 1 if god[1] in team_1 else 0, gods))
    gods_team_2 = list(map(lambda god: 1 if god[1] in team_2 else 0, gods))
    return gods_team_1, gods_team_2

def get_god_selection_names(gods_selection):
    teams = np.split(gods_selection, 2)
    
    team_1 = []
    team_2 = []
    for i in range(len(teams[0])):
        if teams[0][i] == 1:
            team_1.append(gods[i][1])
    for i in range(len(teams[1])):
        if teams[1][i] == 1:
            team_2.append(gods[i][1])
            
    return team_1, team_2

def predict(features):
    # make win probability prediction with the model
    
    # DNN
    prediction = model_dnn.predict([features])
    
    # random forest
    # prediction = model_rf.predict([features])[0]
    
    #print(prediction)
    return prediction

In [10]:
def generate_genome(preteam_1, preteam_2):
    # onehot encode of already chosen characters
    preteam_1_ones = np.where(preteam_1 == 1)[0].tolist()
    preteam_2_ones = np.where(preteam_2 == 1)[0].tolist()

    # generate teams equal to preteams
    team_1 = np.copy(preteam_1)
    team_2 = np.copy(preteam_2)

    # generate a list of possible character choices avoiding duplicates
    possible_ones = np.setdiff1d(np.arange(0, len(preteam_1)), preteam_1_ones)
    possible_ones = np.setdiff1d(possible_ones, preteam_2_ones)

    # randomly select characters to fill the teams
    chosen_ones = sample([*possible_ones], k=10-len(preteam_1_ones)-len(preteam_2_ones))
    team_1[chosen_ones[0:5-len(preteam_1_ones)]] = 1
    team_2[chosen_ones[5-len(preteam_1_ones):]] = 1
    
    # return both teams as onehot encode
    return np.append(team_1, team_2)

def generate_population(preteam_1, preteam_2, size=10):
    # generate `size` genomes in a population
    return [generate_genome(preteam_1, preteam_2) for _ in range(size)]

def fitness(genome):
    # calculate synergy and counter features for given genome
    teams = np.split(genome, 2)
    synergy, counter = get_synergy_counter(teams[0].tolist(), teams[1].tolist())
    genome = np.append(genome, synergy)
    genome = np.append(genome, counter)
    
    # use prediction model to get win probability for given character selection
    return predict(genome.tolist())

def selection_pair(population, fitness_func):
    # randomly select 2 genomes from the population weighted by genomes fitness score
    return choices(
        population=population,
        weights=[fitness(genome) for genome in population],
        k=2
    )

def single_point_crossover(a, b, team_1):
    # split genomes a and b into teams
    a_teams = np.split(a, 2)
    b_teams = np.split(b, 2)
    
    # calculate mandatory ones
    preteam_1_ones = np.where(team_1 == 1)[0].tolist()
    
    # calculate indices of ones in 1st team of each genome
    a_ones = np.where(a_teams[0] == 1)[0].tolist()
    b_ones = np.where(b_teams[0] == 1)[0].tolist()
    
    # define ones that can be changed
    a_ones_changable = np.setdiff1d(a_ones, preteam_1_ones)
    b_ones_changable = np.setdiff1d(a_ones, preteam_1_ones)
    
    # calculate ones unique to genome a and ones unique to genome b that can be changed
    a_only_ones = shuffle(list(set(a_ones_changable) - set(b_ones_changable)))
    b_only_ones = shuffle(list(set(b_ones_changable) - set(a_ones_changable)))
    
    # if 1st team from genome a has any different ones than 1st team from genome b execute crossover
    if len(a_only_ones) > 0:
        
        # calculate random number of ones to switch
        max_num_ones_changable = len(a_only_ones) if len(a_only_ones) < len(b_only_ones) else len(b_only_ones)
        p = randint(1, max_num_ones_changable)
        
        # switch 0 and 1 in 1st teams of genomes
        for i in range(p):
            a_teams[0][a_only_ones[i]] = 0
            b_teams[0][a_only_ones[i]] = 1

            a_teams[0][b_only_ones[i]] = 1
            b_teams[0][b_only_ones[i]] = 0
    
    # return new genomes
    return np.append(a_teams[0], a_teams[1]), np.append(b_teams[0], b_teams[1])

def mutation(genome, team_1, team_2, num=1):
    # split genome into teams
    teams = np.split(genome, 2)

    # calculate indices of mandatory ones and 2nd team ones
    preteam_1_ones = np.where(team_1 == 1)[0].tolist()
    team_2_ones = np.where(teams[1] == 1)[0].tolist()
    
    # calculate indices of ones and zeros in 1st team
    ones = np.where(teams[0] == 1)[0].tolist()
    zeros = np.where(teams[0] == 0)[0].tolist()
    
    # define ones and zeros that can be changed
    ones_changable = np.setdiff1d(ones, preteam_1_ones)
    zeros_changable = np.setdiff1d(zeros, team_2_ones)

    # execute `num` mutations on 1st team
    max_num_ones_changable = num if len(ones_changable) > num else len(ones_changable)
    for _ in range(max_num_ones_changable):

        # change random 1 into 0
        index = choice(ones_changable)
        teams[0][index] = 0

        # change random 0 into 1
        index = choice(zeros_changable)
        teams[0][index] = 1
        
    # return mutated genome
    return np.append(teams[0], teams[1])

In [ ]:
def run_evolution(populate_func,
                  fitness_func,
                  fitness_limit,
                  selection_func,
                  crossover_func,
                  mutation_func,
                  preteam_1,
                  preteam_2,
                  generation_limit=100):
    
    # generate population
    population = populate_func(preteam_1, preteam_2)
    
    # generations loop
    for i in range(generation_limit):
        
        # sort population by fitness ascending, closer to 0 means 1st team wins
        population = sorted(
            population,
            key=lambda genome: fitness_func(genome)
        )
        
        # break the loop if top genome already reached fitness limit
        if fitness_func(population[0]) <= fitness_limit:
            break
        
        # add top 2 genomes into the next generation
        next_generation = population[0:2]
        
        # repeat for half of number of genomes in population, to create an equally large new generation
        for j in range(int(len(population) / 2) - 1):
            
            # select parents and create offspring
            parents = selection_func(population, fitness_func)
            offspring_a, offspring_b = crossover_func(parents[0], parents[1], preteam_1)
            offspring_a = mutation_func(offspring_a, preteam_1, preteam_2)
            offspring_b = mutation_func(offspring_b, preteam_1, preteam_2)
            
            # add their offsprings into the next generation
            next_generation += [offspring_a, offspring_b]
        
        print(f"generation's best: {fitness_func(population[0])[0][0]}")
        population = next_generation
        
    # sort population by fitness ascending
    population = sorted(
        population,
        key=lambda genome: fitness_func(genome)
    )

    # return population and number of generations
    return population, i, fitness_func(population[0])

In [20]:
preteam_1, preteam_2 = get_god_selection(['Achilles', 'Ares'],
                                         ['Kuzenbo', 'Athena'])

population, generations, best_win_prob = run_evolution(
    populate_func=generate_population,
    fitness_func=fitness,
    fitness_limit=0.15,
    selection_func=selection_pair,
    crossover_func=single_point_crossover,
    mutation_func=mutation,
    preteam_1=np.array(preteam_1),
    preteam_2=np.array(preteam_2),
    generation_limit=100
)

print(f"number of generations: {generations}")
print(f"best solution: {get_god_selection_names(population[0])}")
print(f"best solution win probability: {best_win_prob}")

generation's best: 0.18830570578575134
generation's best: 0.18830570578575134
generation's best: 0.18830570578575134
generation's best: 0.18830570578575134
generation's best: 0.15134570002555847
number of generations: 5
best solution: (['Achilles', 'Ares', 'Hera', 'Poseidon', 'Ratatoskr'], ['Athena', 'Kuzenbo', 'Nu Wa', 'Sylvanus', 'Terra'])
best solution win probability: [[0.11887112]]
